# Epic Seven Gear Simulator

This is an ongoing project to create an app in python that will simulate the generation of items (better known as gear in the community), same as those found in the mobile game Epic Seven. This particular notebook is used for writing the test functions before moving them to their respective modules.

In [ ]:
import json
import random
import importlib

import sys
sys.path.append("./data")
sys.path.append("./tests")
sys.path.append("./src")

from src.validation_utils import *
from src.utilities import *
import src.stats as st
import src.gear as gr
#from tests.test_class_gear_v1 import Gear

# importlib.import_module('src.validation_utils')
# importlib.import_module('src.stats')
# importlib.import_module('tests.test_functions')

In [ ]:
importlib.reload(gr)
importlib.reload(st)

In [ ]:
# Import Data
import json

with open('data/types.json', 'r') as types_file:
    TYPES = json.load(types_file)
with open('data/sets.json', 'r') as sets_file:
    SETS = json.load(sets_file)
with open('data/tiers.json', 'r') as tiers_file:
    TIERS = json.load(tiers_file)
with open('data/grades.json', 'r') as grades_file:
    GRADES = json.load(grades_file)
with open('data/stats.json', 'r') as stats_file:
    STATS = json.load(stats_file)

In [ ]:
def add_new_substat(item, stat_id=None, verbose=False):

    if len(item['substats']) < GRADES[item['grade']]['max_substats']:

        # Get list of current stats, both main and substats
        current_stats = item['substats'] + item['mainstat']

        # Get item type:
        type_ = item['type']
        allowed_subs = convert_int_to_string(TYPES[type_]['substat'])

        if stat_id is not None:

            if check_valid_subs(
                    stat_id):  # Check if the provided stat id is valid

                if str(stat_id) not in allowed_subs:
                    print("This substat cannot be added to this item!")
                    return item

                # Check if provided stat is already in the current pool
                elif any(int(stat_id) == s['id'] for s in current_stats):
                    print("Stat already exists in the item!")
                    return item

                else:
                    new_stat = get_stat_by_id(int(stat_id))

        else:
            new_stat = get_non_overlapping_stat(
                current_stats, 'substat', item['type'])

        # Parse the new stats
        parsed_new_stat = parse_stat(
            new_stat, 'substat', item['grade'], item['level'])
        item['substats'].append(parsed_new_stat)

        # Print Confirmation
        if verbose:
            text = format_stat(parsed_new_stat)
            print(f'New Substat Added: {text}!')
            print('---')

    else:
        print('Item already has max number of substats!')

    return item

In [ ]:
def enhance_random_substat(item, verbose=False):
    if 'substats' in item and item['substats']:
        # Choose one of the substats to enhance
        random_substat = random.choice(item['substats'])
        # Get the id of this chosen substat
        random_substat_id = str(random_substat['id'])

        # Get possible roll values along with their rates for this substat
        substat_values = STATS[random_substat_id]['vars']['substat']['values'][item['grade']][item['tier'] - 5]
        substat_rates = STATS[random_substat_id]['vars']['substat']['rates'][item['grade']][item['tier'] - 5]

        try:
            # Get a random value based on the rates
            enhance_value = random.choices(substat_values, substat_rates)[0]
        except IndexError:
            # Handle the case where random.choices doesn't return a value
            return False

        # Add this value to our chosen random substat
        for s in item['substats']:
            if random_substat_id == str(s['id']):
                s['values'][0][1] += enhance_value
                s['rolled'] += 1
                s['reforge_increase'] = get_reforge_increase(
                    random_substat_id, 'substat', s['rolled'])

                # Print Confirmation
                if verbose:
                    text = '+' + \
                        STATS[random_substat_id]['text'] + ' Increase!'
                    text = text.replace(
                        '<A>', str(enhance_value)).replace(
                        '<B>', '')
                    print(text)

                return True
    return False


def enhance_item(item, verbose=False):

    # Current enhance level
    enhance = item['enhance']

    if verbose:
        if enhance == 14:
            print('Max enhance level reached!')

    if enhance < 15:  # This is the max enhance level

        # Upgrade Main Stat
        enhance_mainstat(item)

        # Substat Add Conditions
        # Normal Gear:
        if item['grade'] == 'Normal' and (
                enhance == 2 or enhance == 5 or enhance == 8 or enhance == 11):
            add_new_substat(item, verbose=verbose)

        # Good Gear:
        elif item['grade'] == 'Good' and (enhance == 5 or enhance == 8 or enhance == 11):
            add_new_substat(item, verbose=verbose)

        # Rare Gear:
        elif item['grade'] == 'Rare' and (enhance == 8 or enhance == 11):
            add_new_substat(item, verbose=verbose)

        # Heroic Gear:
        elif item['grade'] == 'Heroic' and (enhance == 11):
            add_new_substat(item, verbose=verbose)

        # For remaining cases
        elif ((enhance + 1) % 3) == 0:  # Enhance only at 3 level increments
            enhance_random_substat(item, verbose=verbose)

        # Increase enhance level by 1
        item['enhance'] += 1

        return item

    else:
        print('Item already at max enhance level!')
        return item

In [ ]:
def reforge_stat(parsed_stat, stat_type='substat'):
    if stat_type == 'substat':
        reforged_value = parsed_stat['values'][0][1] + \
            parsed_stat['reforge_increase']
    else:
        reforged_value = parsed_stat['reforge_increase']
    return reforged_value

In [ ]:
def reforge_item(item):

    if item['is_reforged']:
        print("Cannot reforge item that has already been reforged.")
        return item

    if item['level'] != 85:
        print("Cannot reforge item that is not iLvl 85.")
        return item

    if item['enhance'] < 15:
        print("Cannot reforge item if it has not been enhanced to +15.")
        return item

    else:

        # Set main stat to reforged stat
        item['mainstat'][0]['values'][0][1] = item['mainstat'][0]['reforge_increase']

        # Set each substats to reforged value
        for s in item['substats']:  # iterate through each substat
            s['values'][0][1] = reforge_stat(s)

        # Set reforged status to True
        item['is_reforged'] = True

        # Change item_level to 90
        item['level'] = 90

        # Print confirmation:
        print("Item has been reforged!")

    return item

In [ ]:
# This item modifies a particular stat in the item, stat number is the
# index number of the stat to be modified

def modify_item(item, stat_index=None, mod_stat_id=None,
                mod_type='greater', verbose=False):

    # Check if item is fully enhanced:
    if item['enhance'] != 15:
        print("Cannot modify item unless it has been fully enhanced to +15.")
        return item

    if (stat_index is not None) and (isinstance(
            stat_index, (str, int))) and (0 < stat_index < 5):
        # Get correct indexing for our code (start from 0)
        stat_index = int(stat_index) - 1
    else:
        print("Please provide a valid integer value between 1 and 4 for the stat index.")
        return item

    # Check if any of the other substats have been modified (they will have 'modded' = True)
    # If so, we cannot proceed with modification, as modification is only
    # allowed on the substat where we've applied one mod
    if any(substat.get('modded', False)
           for i, substat in enumerate(item['substats']) if i != stat_index):
        print("Cannot modify substat when another substat has been modded.")
        return item

    # Check if correct stat_index has been provided:
    if (mod_stat_id is not None) and (isinstance(mod_stat_id, (int, str))) and (
            0 <= int(mod_stat_id) <= len(STATS) - 1):
        # Get correct indexing for our code (start from 0)
        mod_stat_id = str(mod_stat_id)
    else:
        print("Please provide an integer value between 0 and 10 for mod_stat_id.")
        return item

    # Check for correct mod_type (it can either be 'greater' or 'lesser')
    if mod_type is not None and mod_type not in ['greater', 'lesser']:
        print("Invalid mod_type provided. Defaulting to Greater Gem.")
        mod_type = 'greater'

    # Get item type and get allowed subs
    type_ = item['type']
    allowed_subs = convert_int_to_string(TYPES[type_]['substat'])

    # Check if the mod stat is applicable in the pool
    if mod_stat_id not in allowed_subs:
        print("This substat cannot be added to this item type")
        return item
    else:
        # Check if the mod stat already exists on the item.
        current_stats = item['substats'] + item['mainstat']

        if any(str(stat['id']) == mod_stat_id for i,
               stat in enumerate(current_stats) if i != stat_index):
            print("Cannot add a substat that already exists on the item!")
            return item
        else:
            # Get the count of how many times this substat has rolled or use 0
            # if it doesn't exist
            roll_count = item['substats'][stat_index].get('rolled', 0)
            # Get a new stat based on provided id
            new_stat = get_stat_by_id(mod_stat_id)
            modified_stat = parse_stat(
                new_stat,
                'substat',
                item['grade'],
                item['level'],
                mod=True,
                rolled=roll_count,
                mod_type=mod_type)  # Parse the stat based on modded values

            # Replace the substat with the new modded stat
            item['substats'][stat_index] = modified_stat
            # Print confirmation
            print("Item has been modded with a new substat!")

    return item

In [ ]:
# def to_view(item):

# 	view_dict = {}
# 	view_dict['name'] = item['name']
# 	view_dict['tier'] = item['tier']
# 	view_dict['type'] = item['type']
# 	view_dict['stats'] = []
# 	view_dict['affixes'] = []
# 	view_dict['legendary'] = []
# 	view_dict['sockets'] = []

# 	for stat in item['stats']:
# 		view_dict['stats'].append(format_affix(stat))
# 	for affix in item['affixes']:
# 		view_dict['affixes'].append(format_affix(affix))
# 	for affix in item['legendary']:
# 		view_dict['legendary'].append(format_affix(affix))
# 	for socket in item['sockets']:
# 		view_dict['sockets'].append(format_affix(socket))

# 	view_dict['main_stat'] = view_dict['stats'].pop(0) #temp
# 	view_dict['sell_value'] = 999
# 	view_dict['durability'] = 100

# 	dmg_icon = 'default'
# 	if item['type'] in CONFIG['offensive']:
# 		dmg_icon = 'offensive'
# 	elif item['type'] in CONFIG['defensive']:
# 		dmg_icon = 'defensive'
# 	view_dict['dmg_icon'] = dmg_icon

# 	return view_dict